# Preprocessing des Donnees pour le Matching Produits - Facteurs d'Emission ADEME

## Objectif
Preparer les donnees pour associer les produits de l'entreprise aux facteurs d'emission de la base ADEME.

**Deux cas distincts (traites separement) :**
- **Cas 1** : Matching avec les Facteurs d'Emission Physiques (FE_ADEME, TYPE='QTE')
- **Cas 2** : Matching avec les Ratios Monetaires (RATIO_MONETAIRE, variable FE.LIB2 uniquement)

In [17]:
import pandas as pd
import numpy as np
import re
import unicodedata
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [18]:
# CONFIGURATION - A MODIFIER SELON VOTRE ENVIRONNEMENT
DATA_INPUT_PATH = Path("./data/input")
DATA_OUTPUT_PATH = Path("./data/preprocessed")
DATA_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
FILE_PRODUITS = DATA_INPUT_PATH / "PRODUITS.xlsx"
FILE_FE_ADEME = DATA_INPUT_PATH / "FE_ADEME.xlsx"
FILE_RATIO_MONETAIRE = DATA_INPUT_PATH / "RATIO_MONETAIRE.xlsx"

In [19]:
# Chargement des donnees
df_produits = pd.read_excel(FILE_PRODUITS, sheet_name='BDD_PRODUITS')
df_fe_ademe = pd.read_excel(FILE_FE_ADEME, sheet_name='data_FE_ADEME')
df_ratio_monetaire = pd.read_excel(FILE_RATIO_MONETAIRE, sheet_name='data RM')

print(f"Produits entreprise : {df_produits.shape}")
print(f"Facteurs emission ADEME : {df_fe_ademe.shape}")
print(f"Ratios monetaires : {df_ratio_monetaire.shape}")

Produits entreprise : (47187, 7)
Facteurs emission ADEME : (14824, 19)
Ratios monetaires : (56, 9)


## Analyse Exploratoire

In [20]:
print("PRODUITS ENTREPRISE")
print(f"Colonnes : {list(df_produits.columns)}")
print(f"\nValeurs manquantes :")
print(df_produits.isnull().sum())
display(df_produits.head())

PRODUITS ENTREPRISE
Colonnes : ['PRODUIT.ID', 'DB.LIB', 'COMPTE.LIB', 'QTE.2022', 'QTE.2023', 'QTE.2024', 'QTE.Total']

Valeurs manquantes :
PRODUIT.ID        0
DB.LIB            0
COMPTE.LIB        0
QTE.2022      27857
QTE.2023      26958
QTE.2024      25937
QTE.Total         0
dtype: int64


,PRODUIT.ID,DB.LIB,COMPTE.LIB,QTE.2022,QTE.2023,QTE.2024,QTE.Total
0,601540,RAI DEPISTAGE (B50) REF RAID2,EFS EXAMENS GREFFES HLA ETMALADIES,35.0,35.0,NaN,70.0
1,601615,SUPPLEMENT DIMANCHE (B20) REF 9004,EFS EXAMENS GREFFES HLA ETMALADIES,NaN,78.0,NaN,78.0
2,246363,RAI DEPISTAGE TIA,EFS EXAMENS GREFFES HLA ETMALADIES,NaN,245.0,NaN,245.0
3,601541,PHENOTYPE RHESUS KELL (B40) REF RHK,EFS EXAMENS GREFFES HLA ETMALADIES,66.0,330.0,NaN,396.0
4,601583,GROUPE SANGUIN ABO RH(D) (B35) REF ABOD,EFS EXAMENS GREFFES HLA ETMALADIES,66.0,330.0,NaN,396.0


In [21]:
print("FACTEURS D'EMISSION ADEME")
print(f"Colonnes : {list(df_fe_ademe.columns)}")
print(f"\nTypes de FE (FE.TYPE) :")
print(df_fe_ademe['FE.TYPE'].value_counts())

# Separation FE physiques
df_fe_physique = df_fe_ademe[df_fe_ademe['FE.TYPE'] == 'QTE'].copy()
print(f"\nFE PHYSIQUES (TYPE='QTE') : {len(df_fe_physique)} lignes")

FACTEURS D'EMISSION ADEME
Colonnes : ['FE.ADEME.ID', 'FE.BDD', 'FE.VAL', 'FE.UNITE', 'FE.Incertitude', 'FE.LIB1', 'FE.LIB2', 'FE.LIB3', 'FE.LIB4', 'FE.LIB5', 'FE.CAT1', 'FE.CAT2', 'FE.CAT3', 'FE.CAT4', 'FE.CAT5', 'FE.DATE-CREA', 'FE.DATE-MODIF', 'FE.DATE-VALID', 'FE.TYPE']

Types de FE (FE.TYPE) :
FE.TYPE
QTE    14502
ECO      322
Name: count, dtype: int64

FE PHYSIQUES (TYPE='QTE') : 14502 lignes


In [22]:
print("RATIOS MONETAIRES")
print(f"Colonnes : {list(df_ratio_monetaire.columns)}")
print(f"\nListe des ratios (FE.LIB2) :")
for i, lib in enumerate(df_ratio_monetaire['FE.LIB2'].values):
    print(f"  {i+1:2d}. {lib}")

RATIOS MONETAIRES
Colonnes : ['FE.RM.ID', 'FE.BDD', 'FE.VAL', 'FE.UNITE', 'FE.Incertitude', 'FE.LIB1', 'FE.LIB2', 'FE.CAT1', 'FE.DATE-VALID']

Liste des ratios (FE.LIB2) :
   1. Agences de voyage, voyagistes, réservations – 2023
   2. Arts, spectacles, musées, bibliothèques / Jeux de hasard – 2023
   3. Assurance, réassurance, retraites (hors sécurité sociale) – 2023
   4. Autres matériels de transport - 2023
   5. Autres produits manufacturés - 2023
   6. Autres produits minéraux non métalliques - 2023
   7. Autres services personnels - 2023
   8. Autres services spécialisés, scientifiques et techniques - 2023
   9. Bois, liège (hors meubles) / Vannerie, sparterie - 2023
  10. Cinéma, TV, musique, enregistrement / Programmation, diffusion – 2023
  11. Commerce de détail, à l'exclusion des automobiles et des motocycles - 2023
  12. Commerce de gros, à l'exclusion des automobiles et des motocycles - 2023
  13. Constructions et travaux de construction - 2023
  14. Edition - 2023
  15. En

In [23]:
print("INCERTITUDE DES FACTEURS D'EMISSION")
print(f"\nFE Physiques :")
print(df_fe_physique['FE.Incertitude'].describe())
print(f"\nRatios Monetaires :")
print(df_ratio_monetaire['FE.Incertitude'].describe())

INCERTITUDE DES FACTEURS D'EMISSION

FE Physiques :
count    14502.000000
mean         0.707753
std          0.366808
min          0.050000
25%          0.300000
50%          1.000000
75%          1.000000
max          4.000000
Name: FE.Incertitude, dtype: float64

Ratios Monetaires :
count    5.600000e+01
mean     8.000000e-01
std      1.120270e-16
min      8.000000e-01
25%      8.000000e-01
50%      8.000000e-01
75%      8.000000e-01
max      8.000000e-01
Name: FE.Incertitude, dtype: float64


## Fonctions de Preprocessing

In [24]:
def normalize_text(text):
    """Normalisation : minuscules, suppression accents et caracteres speciaux."""
    if pd.isna(text) or text is None:
        return ""
    text = str(text).lower().strip()
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_product_description(text):
    """Nettoyage des descriptions produits : suppression refs, codes, etc."""
    if pd.isna(text) or text is None:
        return ""
    text = str(text)
    patterns = [
        r'ref\s*[:\s]*[a-z0-9\-\_\.]+',
        r'ref\s+constr[:\s]*[a-z0-9\-\_\.]+',
        r'ref\s+fourn[:\s]*[a-z0-9\-\_\.]+',
        r'ref\s+fab[:\s]*[a-z0-9\-\_\.]+',
        r'#\d+',
        r'\d{6,}',
        r'\b[a-z]{1,3}\d{4,}[a-z0-9]*\b',
        r'devis\s*n[^a-z]*[a-z0-9\-]+',
        r'substitution\s+ok[^,]*',
        r'gratuit[e]?\s+sur\s+marche',
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text, flags=re.IGNORECASE)
    return text

def extract_keywords(text):
    """Extraction des mots-cles significatifs."""
    stopwords = {
        'le', 'la', 'les', 'un', 'une', 'des', 'de', 'du', 'au', 'aux',
        'et', 'ou', 'en', 'pour', 'par', 'sur', 'avec', 'sans', 'dans',
        'ce', 'cette', 'ces', 'son', 'sa', 'ses', 'leur', 'leurs',
        'qui', 'que', 'quoi', 'dont',
        'est', 'sont', 'a', 'ont', 'ete', 'etre', 'avoir',
        'plus', 'moins', 'tres', 'bien', 'tout', 'tous', 'toute', 'toutes',
        'unite', 'unites', 'paire', 'paires', 'lot', 'lots', 'boite', 'boites',
        'ref', 'constr', 'fourn', 'fab',
        'cm', 'mm', 'ml', 'kg', 'gr', 'mg', 'l', 'm', 'x', 'n', 'v', 'd',
    }
    text = normalize_text(text)
    words = text.split()
    keywords = [w for w in words if len(w) > 2 and w not in stopwords and not w.isdigit()]
    return ' '.join(keywords)

def clean_ademe_lib(lib1, lib2, lib3):
    """Concatenation et nettoyage des libelles ADEME."""
    parts = []
    for lib in [lib1, lib2, lib3]:
        if pd.notna(lib) and str(lib).strip():
            text = str(lib)
            text = re.sub(r'"+', ' ', text)
            text = text.strip(',').strip()
            if text and text.lower() != 'nan':
                parts.append(text)
    return re.sub(r'\s+', ' ', ' '.join(parts)).strip()

def clean_ratio_monetaire_lib(lib2):
    """Nettoyage du libelle FE.LIB2 des ratios monetaires."""
    if pd.isna(lib2) or lib2 is None:
        return ""
    text = str(lib2)
    text = re.sub(r'[\-\u2013]\s*\d{4}\s*$', '', text)
    return text.strip()

## Preprocessing des Produits Entreprise

In [25]:
df_produits_clean = df_produits.copy()
df_produits_clean['description_clean'] = df_produits_clean['DB.LIB'].apply(clean_product_description)
df_produits_clean['description_normalized'] = df_produits_clean['description_clean'].apply(normalize_text)
df_produits_clean['keywords'] = df_produits_clean['description_clean'].apply(extract_keywords)

print(f"Preprocessing produits termine : {len(df_produits_clean)} lignes")

Preprocessing produits termine : 47187 lignes


In [26]:
print("Exemples de preprocessing :")
for i in [0, 10, 50, 100, 200]:
    if i < len(df_produits_clean):
        row = df_produits_clean.iloc[i]
        print(f"\n[{i}] Original: {row['DB.LIB'][:80]}...")
        print(f"    Keywords: {row['keywords']}")

Exemples de preprocessing :

[0] Original: RAI DEPISTAGE (B50) REF RAID2...
    Keywords: rai depistage b50

[10] Original: RETINOMAX PRINTER 5...
    Keywords: retinomax printer

[50] Original: PLANCHE DE TRANSFERT AMAGNETIQUEROLLBO DY RIGIDE MINI 90*50CMREF CONSTR ROLBO DY...
    Keywords: planche transfert amagnetiquerollbo rigide mini 50cm rolbo

[100] Original: COUSSIN UNIVERSEL PETIT FORMAT 30X40NE G0434 (LOT DE 2)...
    Keywords: coussin universel petit format 30x40ne

[200] Original: COUSSIN DISTAL SILICONE 80MM ST041REV8  (UNITE)...
    Keywords: coussin distal silicone 80mm st041rev8


In [27]:
print("\nStatistiques :")
print(f"Produits total : {len(df_produits_clean)}")

print(f"description vide : {(df_produits_clean['description_normalized'] == '').sum()}")
print(f"Longueur moyenne description : {df_produits_clean['description_normalized'].str.split().str.len().mean():.1f} mots")

print(f"Keywords vides : {(df_produits_clean['keywords'] == '').sum()}")
print(f"Longueur moyenne keywords : {df_produits_clean['keywords'].str.split().str.len().mean():.1f} mots")

#df_produits_clean = df_produits_clean[df_produits_clean['keywords'] != '']

print(f"Keywords vides : {(df_produits_clean['keywords'] == '').sum()}")
print(f"Longueur moyenne keywords : {df_produits_clean['keywords'].str.split().str.len().mean():.1f} mots")




Statistiques :
Produits total : 47187
description vide : 2
Longueur moyenne description : 8.4 mots
Keywords vides : 7
Longueur moyenne keywords : 5.1 mots
Keywords vides : 7
Longueur moyenne keywords : 5.1 mots


## Preprocessing des Facteurs d'Emission Physiques (CAS 1)

In [28]:
df_fe_phys_clean = df_fe_physique.copy()
df_fe_phys_clean['description_raw'] = df_fe_phys_clean.apply(
    lambda row: clean_ademe_lib(row['FE.LIB1'], row['FE.LIB2'], row['FE.LIB3']), axis=1)
df_fe_phys_clean['description_normalized'] = df_fe_phys_clean['description_raw'].apply(normalize_text)
df_fe_phys_clean['keywords'] = df_fe_phys_clean['description_raw'].apply(extract_keywords)

print(f"Preprocessing FE physiques termine : {len(df_fe_phys_clean)} lignes")

Preprocessing FE physiques termine : 14502 lignes


In [29]:
print("Exemples FE physiques :")
for i in range(min(5, len(df_fe_phys_clean))):
    row = df_fe_phys_clean.iloc[i]
    print(f"\n[{i}] ID: {row['FE.ADEME.ID']}")
    print(f"    Combined: {row['description_raw'][:70]}...")
    print(f"    Keywords: {row['keywords']}")

Exemples FE physiques :

[0] ID: 34052
    Combined: Agribalyse, Salade César au poulet (salade verte,entrées et plats comp...
    Keywords: agribalyse salade cesar poulet salade verte entrees plats composes salade cesar poulet salade verte fromage croutos sauce

[1] ID: 33558
    Combined: produits céréaliers, Agribalyse, Brioche fourrée crème pâtissière (typ...
    Keywords: produits cerealiers agribalyse brioche fourree creme patissiere type chinois viennoiseries brioche fourree creme patissiere type chinois preemballee

[2] ID: 32682
    Combined: produits céréaliers, Agribalyse,biscuits sucrés , Goûter sec fourré ( ...
    Keywords: produits cerealiers agribalyse biscuits sucres gouter sec fourre sandwiche parfum chocolat gouter sec fourre sandwiche parfum chocolat

[3] ID: 32683
    Combined: produits céréaliers, Agribalyse,biscuits sucrés , Goûter sec fourré ( ...
    Keywords: produits cerealiers agribalyse biscuits sucres gouter sec fourre sandwiche parfum fruits gouter sec fo

In [30]:
print("\nStatistiques FE physiques :")
print(f"Total : {len(df_fe_phys_clean)}")
print(f"Keywords vides : {(df_fe_phys_clean['description_normalized'] == '').sum()}")
print(f"Keywords vides : {(df_fe_phys_clean['keywords'] == '').sum()}")
print(f"\nDistribution CAT1 :")
print(df_fe_phys_clean['FE.CAT1'].value_counts())


Statistiques FE physiques :
Total : 14502
Keywords vides : 0
Keywords vides : 7

Distribution CAT1 :
FE.CAT1
Achats de biens                   8722
Réseaux de chaleur / froid        2510
Electricité                        818
Combustibles                       728
Transport de marchandises          463
Transport de personnes             354
Process et émissions fugitives     318
Traitement des déchets             312
UTCF                               197
Statistiques territoriales          60
Achats de services                  20
Name: count, dtype: int64


In [31]:
df_fe_phys_clean = df_fe_phys_clean[df_fe_phys_clean['keywords'] != ' ']


## Preprocessing des Ratios Monetaires (CAS 2)

In [32]:
df_rm_clean = df_ratio_monetaire.copy()
df_rm_clean['description_raw'] = df_rm_clean['FE.LIB2'].apply(clean_ratio_monetaire_lib)
df_rm_clean['description_normalized'] = df_rm_clean['description_raw'].apply(normalize_text)
df_rm_clean['keywords'] = df_rm_clean['description_raw'].apply(extract_keywords)

print(f"Preprocessing ratios monetaires termine : {len(df_rm_clean)} lignes")

Preprocessing ratios monetaires termine : 56 lignes


In [33]:
print("Ratios monetaires preprocesses :")
for i in range(len(df_rm_clean)):
    row = df_rm_clean.iloc[i]
    print(f"\n[{i+1:2d}] ID: {row['FE.RM.ID']}")
    print(f"     Original: {row['FE.LIB2']}")
    print(f"     Cleaned : {row['description_raw']}")
    print(f"     Keywords: {row['keywords']}")

Ratios monetaires preprocesses :

[ 1] ID: 43515
     Original: Agences de voyage, voyagistes, réservations – 2023
     Cleaned : Agences de voyage, voyagistes, réservations
     Keywords: agences voyage voyagistes reservations

[ 2] ID: 43460
     Original: Arts, spectacles, musées, bibliothèques / Jeux de hasard – 2023
     Cleaned : Arts, spectacles, musées, bibliothèques / Jeux de hasard
     Keywords: arts spectacles musees bibliotheques jeux hasard

[ 3] ID: 43475
     Original: Assurance, réassurance, retraites (hors sécurité sociale) – 2023
     Cleaned : Assurance, réassurance, retraites (hors sécurité sociale)
     Keywords: assurance reassurance retraites hors securite sociale

[ 4] ID: 43300
     Original: Autres matériels de transport - 2023
     Cleaned : Autres matériels de transport
     Keywords: autres materiels transport

[ 5] ID: 43365
     Original: Autres produits manufacturés - 2023
     Cleaned : Autres produits manufacturés
     Keywords: autres produits manufa

## Export des Fichiers

In [34]:
# Selection des colonnes
cols_produits = ['PRODUIT.ID', 'DB.LIB', 'COMPTE.LIB', 
                 'QTE.2022', 'QTE.2023', 'QTE.2024', 'QTE.Total',
                 'description_clean', 'description_normalized', 'keywords']

cols_fe_phys = ['FE.ADEME.ID', 'FE.VAL', 'FE.UNITE', 'FE.Incertitude',
                'FE.LIB1', 'FE.LIB2', 'FE.LIB3', 'FE.CAT1', 'FE.CAT2',
                'description_raw', 'description_normalized', 'keywords']

cols_rm = ['FE.RM.ID', 'FE.VAL', 'FE.UNITE', 'FE.Incertitude',
           'FE.LIB2', 'FE.CAT1',
           'description_raw', 'description_normalized', 'keywords']

df_produits_export = df_produits_clean[cols_produits].copy()
df_fe_phys_export = df_fe_phys_clean[cols_fe_phys].copy()
df_rm_export = df_rm_clean[cols_rm].copy()

In [35]:
print("RESUME DES FICHIERS")
print(f"\n1. Produits : {len(df_produits_export)} lignes")
print(f"2. FE Physiques (CAS 1) : {len(df_fe_phys_export)} lignes")
print(f"3. Ratios Monetaires (CAS 2) : {len(df_rm_export)} lignes")

RESUME DES FICHIERS

1. Produits : 47187 lignes
2. FE Physiques (CAS 1) : 14502 lignes
3. Ratios Monetaires (CAS 2) : 56 lignes


In [36]:
# Export CSV
df_produits_export.to_csv(DATA_OUTPUT_PATH / 'produits_preprocessed.csv', index=False, encoding='utf-8')
df_fe_phys_export.to_csv(DATA_OUTPUT_PATH / 'fe_physique_preprocessed.csv', index=False, encoding='utf-8')
df_rm_export.to_csv(DATA_OUTPUT_PATH / 'ratio_monetaire_preprocessed.csv', index=False, encoding='utf-8')

print("Fichiers exportes :")
print(f"  - {DATA_OUTPUT_PATH / 'produits_preprocessed.csv'}")
print(f"  - {DATA_OUTPUT_PATH / 'fe_physique_preprocessed.csv'}")
print(f"  - {DATA_OUTPUT_PATH / 'ratio_monetaire_preprocessed.csv'}")

Fichiers exportes :
  - data/preprocessed/produits_preprocessed.csv
  - data/preprocessed/fe_physique_preprocessed.csv
  - data/preprocessed/ratio_monetaire_preprocessed.csv


## Verification Qualite

In [37]:
# Doublons de keywords
duplicates = df_produits_export[df_produits_export.duplicated(subset=['keywords'], keep=False)]
duplicates_non_empty = duplicates[duplicates['keywords'] != '']
print(f"Produits avec keywords identiques : {len(duplicates_non_empty)}")

# Keywords vides
kw_vides = df_produits_export[df_produits_export['keywords'] == '']
print(f"Produits avec keywords vides : {len(kw_vides)}")
if len(kw_vides) > 0:
    for _, row in kw_vides.iterrows():
        print(f"  - {row['DB.LIB']}")

Produits avec keywords identiques : 23323
Produits avec keywords vides : 7
  -  
  - M1224D ##16/03/2022
  - REFERENCES 11996-000017
  - 96
  - REF 2206488/1828473
  - CQ BF
  - BPK1520 65777 ID


In [38]:

print(f"""
Fichiers generes dans {DATA_OUTPUT_PATH} :
  1. produits_preprocessed.csv ({len(df_produits_export)} lignes)
  2. fe_physique_preprocessed.csv ({len(df_fe_phys_export)} lignes) - CAS 1
  3. ratio_monetaire_preprocessed.csv ({len(df_rm_export)} lignes) - CAS 2

""")


Fichiers generes dans data/preprocessed :
  1. produits_preprocessed.csv (47187 lignes)
  2. fe_physique_preprocessed.csv (14502 lignes) - CAS 1
  3. ratio_monetaire_preprocessed.csv (56 lignes) - CAS 2


